In [2]:
import pandas as pd


In [32]:
products = pd.read_csv("data/products.csv")
store_purchases = pd.read_csv("data/store_purchases.csv")
users = pd.read_csv("data/users.csv")
web_visits = pd.read_csv("data/web_visits.csv")

In [33]:
store_user = store_purchases.merge(users, how="left", left_on = ["user_id"], right_on = ["user_ID"])
store_user.drop(["user_ID"], inplace=True, axis=1)
store_user

,user_id,date,family,class,type,product,product_id,price,card_owner
0,49263,2020-02-26,family_6,class_3,type_4,product_3,00043a7a-ba01-4653-a8c9-dfc1c256372d,5.0,False
1,30940,2022-12-16,family_6,class_3,type_4,product_3,00043a7a-ba01-4653-a8c9-dfc1c256372d,5.0,False
2,11386,2020-02-18,family_6,class_3,type_4,product_3,00043a7a-ba01-4653-a8c9-dfc1c256372d,5.0,False
3,37749,2020-11-04,family_6,class_3,type_4,product_3,00043a7a-ba01-4653-a8c9-dfc1c256372d,5.0,False
4,14128,2021-02-10,family_6,class_3,type_4,product_3,00043a7a-ba01-4653-a8c9-dfc1c256372d,5.0,False
...,...,...,...,...,...,...,...,...,...
171648,34856,2021-08-08,family_6,class_1,type_1,product_12,6bcfe8b4-42c2-4089-bccc-b670ad8210d6,152.0,False
171649,47325,2022-10-21,family_6,class_1,type_1,product_12,6bcfe8b4-42c2-4089-bccc-b670ad8210d6,152.0,False
171650,30303,2022-05-12,family_6,class_1,type_1,product_12,6bcfe8b4-42c2-4089-bccc-b670ad8210d6,152.0,False
171651,20029,2021-01-17,family_6,class_1,type_1,product_12,6bcfe8b4-42c2-4089-bccc-b670ad8210d6,152.0,False


In [45]:
df = web_visits.merge(store_user[["user_id", "product_id", "date", "card_owner"]], how = "left", on=["user_id", "product_id"])
df

MemoryError: Unable to allocate 207. MiB for an array with shape (27195505,) and data type int64

In [39]:
df["offline_conversion"] = 0

,family_x,class_x,type_x,web_visit_date,user_id,product_x,price_x,product_id,conversion,date,family_y,class_y,type_y,product_y,price_y,card_owner
0,family_9,class_8,type_2,2022-03-02,1770.0,product_17,103,567e07ef-45e3-406b-a2b1-1175187504be,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,family_4,class_8,type_2,2022-04-08,43333.0,product_30,1,5f7dcb42-5eb4-4aaf-bd36-40e766fc7449,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,family_9,class_8,type_1,2022-03-14,33025.0,product_16,2,6d2ac9e4-9baf-4338-b3d9-671bcdcb36eb,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,family_4,class_8,type_2,2022-04-25,24377.0,product_22,34,717b0355-71aa-4be6-9eff-afa5c7ec95c7,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,family_4,class_8,type_2,2022-09-19,40274.0,product_32,1,75b6a327-af04-413a-a605-a07cc66b3fb4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27238191,family_2,class_1,type_1,2020-06-17,18550.0,product_19,163,1b002490-5f32-4482-8a50-83379e511519,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27238192,family_2,class_1,type_1,2020-06-13,27973.0,product_16,1,20b90b1a-beeb-4308-82ba-7c55121141d8,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27238193,family_2,class_1,type_1,2022-02-07,39666.0,product_16,1,20b90b1a-beeb-4308-82ba-7c55121141d8,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27238194,family_5,class_1,type_2,2022-05-09,28123.0,product_5,166,21c5eef8-2ffe-4dbf-9b9a-04f87715e6af,0,2021-12-06,family_5,class_1,type_2,product_5,166.0,False
